## Trained LAT-AI model
Author: Konrad Pieszko
Date "2022-07-29"






In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import tqdm
import os
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
import random
import pickle


seed=42
random.seed(seed)


target="TEE_base_clot_observed"


oversampling=0.1   # these values were chosen so that they represent the prevalence of positive class of about 0.05, 
undersampling=0.5  # but accordint to imbalanced learn documentations, bot under and oversampling should be used for opitmal reuslts

                            
#fixed parameters based on parameter space search using STREMLINE ML
params = {'max_depth': 2, 'eta': 1.780190e-03,'objective':'binary:logistic'}
params['nthread'] = 32
params['eval_metric'] = 'auc'
params['min_child_weight'] = 3.839629e-01
params['colsample_bytree'] = 4.099086e-01
params['subsample'] = 5.697469e-01
params['gamma'] = 1.989277e-06
params['min_samples_leaf']=30
params['n_estimators']=438
params["tree_method"]="hist"   
params["verbosity"]=0
params["random_state"]=seed






/opt/homebrew/Caskroom/miniforge/base/envs/streamline/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Import data

In [11]:
training_data = pd.read_csv('./sample_data_LATAI.csv')

X = training_data.drop(target, axis=1)
X= X.apply(pd.to_numeric)
y = training_data[target].astype(int)

X

,AF,AFL,arrythmia_duration,EHRA,Age,Sex,BMI,HF_status,NYHA,MS,...,INR,APTT,WBC,LA_dimension,EF,LA_area,LAVI,TEE_base_rythm,base_OAC_chronic,on_VKA
0,1,0,3,2,71,2,28,2,1,3,...,NaN,NaN,NaN,42,58,47.0,50,1,0,0
1,1,0,2,4,42,2,25,1,1,3,...,1.1,39.2,7.4,48,54,37.7,51,0,1,0
2,0,1,3,5,49,2,37,2,1,3,...,2.6,48.0,7.4,48,50,29.0,35,2,1,0
3,1,0,3,2,55,1,24,2,1,3,...,3.6,55.6,8.3,47,30,30.0,61,0,1,1
4,1,0,3,1,75,1,28,4,4,3,...,1.2,41.4,7.9,35,60,26.0,35,0,1,0


#  Final model training on the entire dataset 
This is just to show the model training steps and hyperparameters. The training data is not provided. The sample data can be used to generate predictions

In [4]:

model = xgb.XGBClassifier(**params)
imp_mean = IterativeImputer(random_state=seed)
X_train_imp=imp_mean.fit_transform(X)  #fit multivariate imputer on traint only   
      #transform the test set using trained imputer
y_train = y


over = SMOTE(sampling_strategy=oversampling, random_state=seed)
under = RandomUnderSampler(sampling_strategy=undersampling, random_state=seed)
steps = [('over', over), ('under', under), ('model', model)]
pipieline = Pipeline(steps=steps)    

pipieline.fit(X_train_imp, y_train)
#model.fit(X_train_imp, y_train)

#pickle_out = open(os.path.join("./LATAI_model.pickle"), 'wb')
#pickle.dump(pipieline,pickle_out)
#pickle_out.close()




# Load the pickled model, get LAT-AI score for an imaginary patient


In [12]:
# load the model from disk
loaded_model = pickle.load(open('./LATAI_model.pickle', 'rb'))

external_data= pd.read_csv("./sample_data_LATAI.csv")
X_external=external_data.drop(target, axis=1)
y_pred=loaded_model.predict_proba(X_external)[:,1]  # no need to impute missing data at the time of prediction!


y_pred



/opt/homebrew/Caskroom/miniforge/base/envs/streamline/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


array([0.38905624, 0.3574141 , 0.42249483, 0.46470425, 0.33836585],
      dtype=float32)